### Traning the LightGBM model with Hyperparameter Tuning

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import mlflow

# Setup MLFlow Tracking Server
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [4]:
# Set or Create an experiment
mlflow.set_experiment("Exp 6- LightGBM with Hyperparameter Tuning")

2025/12/26 22:12:39 INFO mlflow.tracking.fluent: Experiment with name 'Exp 6- LightGBM with Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e9775ed4cf9a44a5b571f121ccf8d944', creation_time=1766767358207, experiment_id='8', last_update_time=1766767358207, lifecycle_stage='active', name='Exp 6- LightGBM with Hyperparameter Tuning', tags={}>

In [6]:
import os

ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import mlflow
import mlflow.sklearn
import optuna
import matplotlib.pyplot as plt
import seaborn as sns

e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df = pd.read_csv('./artifacts/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
print(df.shape)
df.sample(5)

(36662, 2)


,clean_comment,category
8221,sintesi reddit hai trovato subumani antiteisti...,-1
27107,holy shit hit park well fucking done need back...,-1
27959,identical setup ordered part magni3 modi 6xx r...,1
2088,prathiba patil hoti kaise bolti,0
16164,surprised see vince throw money money gain gen...,1


In [9]:
# For LightGBM, remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Remove the rows where target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)
max_features = 1000

# # Split the data into training and test datasets
# X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']


smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [10]:
# Function to log results in MLFlow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run() as run:
        # Log model type and trial number
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log Accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("Accuracy: ", accuracy)

        # Log Classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    if metric != "support":
                        mlflow.log_metric(f"{label}_{metric}", value)

        # Log Confusion Matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Model={model_name}")

        # Save and log the confusion matrix plot
        filename = f"Exp6_TF_IDF_Trigrams_{model_name}_ConfusionMatrix.png"
        cm_path = os.path.join(ARTIFACT_DIR, filename)
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_Model_TF_IDF_Trigrams_SMOTE")


In [11]:
# Optuna objective function for LightGBM
def objective_lightgbm(trial):
    # Hyperparameter space to explore
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 15)
    num_leaves = trial.suggest_int("num_leaves", 20, 150)
    min_child_samples = trial.suggest_int("min_child_samples", 10, 100)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
    subsample = trial.suggest_float("subsample", 0.5, 1.0)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 10.0, log=True)      # L1 Regularization
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 10.0, log=True)    # L2 Regularization

    # Log trial parameters
    params = {
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "num_leaves": num_leaves,
        "min_child_samples": min_child_samples,
        "colsample_bytree": colsample_bytree,
        "subsample": subsample,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda
    }

    # Create and train LightGBM model
    model = LGBMClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        num_leaves=num_leaves,
        min_child_samples=min_child_samples,
        colsample_bytree=colsample_bytree,
        subsample=subsample,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
    )
    
    # Log each trial as a separate run in MLFLow
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)

    return accuracy

In [12]:
# Run Optuna for LightGBM, log the best model and plot the importance of each parameter
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=100)    # Increased to 100 trials

    # Get the best parameters
    best_params = study.best_params
    best_model = LGBMClassifier(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        num_leaves=best_params['num_leaves'],
        min_child_samples=best_params['min_child_samples'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        random_state=42
    )

    # Log the best model with MLFlow and print the classification report
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    # Plot the parameter importance
    optuna.visualization.plot_param_importances(study).show()

    # Plot optimization history
    optuna.visualization.plot_optimization_history(study).show()

In [ ]:
# Run the experiment for LightGBM with HPT
run_optuna_experiment()

[I 2025-12-26 23:09:00,820] A new study created in memory with name: no-name-dd4d482b-1b97-4fe1-a44f-026ddaec7ae3
2025/12/26 23:11:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
[W 2025-12-26 23:11:45,150] Trial 0 failed with parameters: {'n_estimators': 290, 'learning_rate': 0.012807504666722966, 'max_depth': 8, 'num_leaves': 136, 'min_child_samples': 63, 'colsample_bytree': 0.9425757564305112, 'subsample': 0.6167925389109709, 'reg_alpha': 0.001117920950454605, 'reg_lambda': 0.00017007743292834712} because of the following error: The value None could not be cast to float..
[W 2025-12-26 23:11:45,152] Trial 0 failed with value None.
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition 